# pycisTopic analysis

Cell downsampled dataset, using consensus peak regions.

In [102]:
import pycisTopic
pycisTopic.__version__

'0.1.dev300+g7494158'

In [103]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [104]:
import pickle
import pandas as pd

In [105]:
import os
wdir = '/lustre1/project/stg_00002/lcb/fderop/data/20211024_hca_benchmark_libds_merged/'
os.chdir( wdir )

In [106]:
import glob
from collections import OrderedDict
filenames = glob.glob('fragments_postbap/*.sinto.mm.fragments.tsv.gz')
samples = [item.replace(".sinto.mm.fragments.tsv.gz", "") for item in filenames]
samples = [item.replace("fragments_postbap/", "") for item in samples]
fragments_dict = {samples[i]: filenames[i] for i in range(len(samples))}
fragments_dict = OrderedDict(sorted(fragments_dict.items()))
fragments_dict.keys()

odict_keys(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'])

In [107]:
fragments_dict

OrderedDict([('Broad_1',
              'fragments_postbap/Broad_1.sinto.mm.fragments.tsv.gz'),
             ('Broad_2',
              'fragments_postbap/Broad_2.sinto.mm.fragments.tsv.gz'),
             ('Broad_mito_1',
              'fragments_postbap/Broad_mito_1.sinto.mm.fragments.tsv.gz'),
             ('Broad_mito_2',
              'fragments_postbap/Broad_mito_2.sinto.mm.fragments.tsv.gz'),
             ('CNAG_1', 'fragments_postbap/CNAG_1.sinto.mm.fragments.tsv.gz'),
             ('CNAG_2', 'fragments_postbap/CNAG_2.sinto.mm.fragments.tsv.gz'),
             ('Sanger_1',
              'fragments_postbap/Sanger_1.sinto.mm.fragments.tsv.gz'),
             ('Sanger_2',
              'fragments_postbap/Sanger_2.sinto.mm.fragments.tsv.gz'),
             ('Stanford_1',
              'fragments_postbap/Stanford_1.sinto.mm.fragments.tsv.gz'),
             ('Stanford_2',
              'fragments_postbap/Stanford_2.sinto.mm.fragments.tsv.gz'),
             ('VIB_1', 'fragments_postbap/VIB_

# load consensus peak dict


load a dictionary with paths to consensus files, generated in notebook 3

In [108]:
import glob
from collections import OrderedDict
consensus_peaks_dict = {}
key = 'libds_merged'
consensus_regions = 'consensus_peak_calling/'+key+'/'+key+'__consensus_regions.bed'


filenames = glob.glob('fragments_postbap/*.sinto.mm.fragments.tsv.gz')
samples = [item.replace(".sinto.mm.fragments.tsv.gz", "") for item in filenames]
samples = [item.replace("fragments_postbap/", "") for item in samples]
consensus_peaks_dict = {samples[i]: consensus_regions for i in range(len(samples))}
consensus_peaks_dict = OrderedDict(sorted(consensus_peaks_dict.items()))
consensus_peaks_dict.keys()

odict_keys(['Broad_1', 'Broad_2', 'Broad_mito_1', 'Broad_mito_2', 'CNAG_1', 'CNAG_2', 'Sanger_1', 'Sanger_2', 'Stanford_1', 'Stanford_2', 'VIB_1', 'VIB_2', 'VIB_Hydrop_1', 'VIB_Hydrop_2', 's3atac'])

## Library QC metrics

In [109]:
from pycisTopic.qc import compute_qc_stats
import pyranges

! cp /staging/leuven/stg_00002/lcb/cflerin/analysis/pbmc_atac/analysis2/data_freeze_Jun2021/atac_qc_multiplet_merged/jupyter/biomart_annot.pickle .

In [110]:
import pickle
f_biomart = '/lustre1/project/stg_00002/lcb/fderop/data/20211013_hca_benchmark_libds/biomart_annot.pickle'
with open(f_biomart, 'rb') as f:
    annot = pickle.load(f)
annot

,Chromosome,Start,Strand,Gene,Transcript_type
8986,chrY,9337464,1,TSPY4,protein_coding
8987,chrY,9337510,1,TSPY4,protein_coding
9024,chrY,22490397,1,PRY,protein_coding
9100,chrY,14056227,1,VCY1B,protein_coding
9136,chrY,12662368,1,USP9Y,protein_coding
...,...,...,...,...,...
236024,chr1,36479519,-1,CSF3R,protein_coding
236025,chr1,36471474,-1,CSF3R,protein_coding
236033,chr1,36482051,-1,CSF3R,protein_coding
236034,chr1,36323645,-1,EVA1B,protein_coding


In [35]:
import ray
ray.shutdown()

metadata_bc_dict, profile_data_dict = compute_qc_stats(
        fragments_dict=fragments_dict,
        tss_annotation=annot,
        stats=['barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'frip'],
        label_list=None,
        path_to_regions=consensus_peaks_dict,
        n_cpu=24,
        valid_bc=None,
        n_frag=100,
        n_bc=None,
        tss_flank_window=2000,
        tss_window=50,
        tss_minimum_signal_window=100,
        tss_rolling_window=10,
        min_norm=0.1,
        remove_duplicates = True,
        )

2021-10-25 17:37:01,868 cisTopic     INFO     n_cpu is larger than the number of samples. Setting n_cpu to the number of samples


2021-10-25 17:37:03,720	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=34977) 2021-10-25 17:37:07,035 cisTopic     INFO     Reading CNAG_2
(pid=34980) 2021-10-25 17:37:07,057 cisTopic     INFO     Reading Broad_2
(pid=34984) 2021-10-25 17:37:07,149 cisTopic     INFO     Reading VIB_2
(pid=34981) 2021-10-25 17:37:07,099 cisTopic     INFO     Reading VIB_Hydrop_2
(pid=34988) 2021-10-25 17:37:07,141 cisTopic     INFO     Reading Sanger_1
(pid=34982) 2021-10-25 17:37:07,068 cisTopic     INFO     Reading Stanford_1
(pid=34979) 2021-10-25 17:37:07,129 cisTopic     INFO     Reading Stanford_2
(pid=34986) 2021-10-25 17:37:07,129 cisTopic     INFO     Reading CNAG_1
(pid=34987) 2021-10-25 17:37:07,149 cisTopic     INFO     Reading s3atac
(pid=34985) 2021-10-25 17:37:07,166 cisTopic     INFO     Reading VIB_Hydrop_1
(pid=34978) 2021-10-25 17:37:07,166 cisTopic     INFO     Reading VIB_1
(pid=34990) 2021-10-25 17:37:07,181 cisTopic     INFO     Reading Broad_mito_1
(pid=34976) 2021-10-25 17:37:07,181 cisTopic     INFO     Reading Sanger_2
(pid=34989) 2021-10-25

In [48]:
f_qc_dir = 'pycistopic_consensus_peaks/qc__consensus_peaks'
if not os.path.exists(os.path.join(wdir, f_qc_dir)):
    os.makedirs(os.path.join(wdir, f_qc_dir))
    

In [49]:
with open(os.path.join(wdir, f_qc_dir, 'metadata.pickle'), 'wb') as f:
    pickle.dump(metadata_bc_dict, f)

with open(os.path.join(wdir, f_qc_dir, 'profile_data.pickle'), 'wb') as f:
    pickle.dump(profile_data_dict, f)

In [119]:
import seaborn as sns